In [10]:
import ray
from ray.job_submission import JobSubmissionClient
import time
import subprocess

# Ray cluster information
ray_head_ip = "kuberay-head-svc.kuberay.svc.cluster.local"
ray_head_port = 8265
ray_address = f"http://{ray_head_ip}:{ray_head_port}"
client = JobSubmissionClient(ray_address)

In [62]:
# Executors
def local_submission(n,m,p):
    print("Local submission:")
    command = ["python", "ray-matrix_multiplication-example.py",n,m,p]
    subprocess.run(command)

def ray_submission(n,m,p,c):
    print("#################################\nRay submission:")
    job_id = c.submit_job(
        entrypoint=f"python ray-matrix_multiplication-example.py {n} {m} {p}",
        runtime_env={
            "working_dir": "./", 
        },
        entrypoint_num_cpus=3
    )

    # Waiting for Ray to finish the job and print the result
    while True:
        status = c.get_job_status(job_id)
        if status in [ray.job_submission.JobStatus.RUNNING, ray.job_submission.JobStatus.PENDING]:
            time.sleep(5)
        else:
            break
    try:
        logs = c.get_job_logs(job_id) 
        print(logs)
    except RuntimeError as e:
        print(f"Failed to get job logs, please check logs on ray dashboard ")

In [63]:
# matrix1 n x m, matrix2 m x p
n = "10000"
m = "10000"
p = "10000"

In [ ]:
# Local
local_submission(n,m,p)

# Ray
ray_submission(n,m,p,client)
ray.shutdown()